# Python 使用 CSS 选择器解析网页

本笔记系统整理了在 Python 中使用 **CSS 选择器 (CSS Selectors)** 提取网页内容的常见方式，涵盖 `parsel` (基于 lxml)、`BeautifulSoup`、`pyquery`、`lxml` 原生、以及高性能的 `selectolax`。同时给出常见选择器模式、技巧、容易踩的坑与性能建议。
```shell
pip install parsel beautifulsoup4 lxml pyquery selectolax
```

## 1. CSS 选择器速览
常见模式 (与浏览器开发者工具一致)：
- `tag` 标签名，如 `div`, `a`, `li`
- `.class` 类名 (单个)；多个类：`.item.active` (与)
- `#id` 唯一 id
- `tag.class` 组合：`div.container`
- 子代 `parent > child`：`ul > li` 仅直接子元素
- 后代 `ancestor descendant`：`div a` 任意深度后代
- 相邻兄弟 `prev + next`；一般兄弟 `prev ~ siblings`
- 属性选择：`a[href]`，`a[href^=https]`，`img[src$=.png]`，`a[data-id*=123]`
- 伪类 (lxml 不支持全部)：`:first-child`, `:nth-child(2)`, `:last-child` 等 (不同库支持度不同)
- 取文本/属性：多数库提供额外 API，不用在选择器里写。

> 与 XPath 对比：CSS 更简洁但在处理复杂层级或条件过滤时 XPath 更强。

## 2. 准备：统一示例 HTML
下面构造一段示例 HTML，重复使用，避免网络不稳定影响演示。

In [4]:
sample_html = '''\n<html>\n  <head><title>Demo Page</title></head>\n  <body>\n    <div id="main" class="container">\n      <h1 class="title">示例标题</h1>\n      <ul class="items">\n        <li class="item active" data-id="101">苹果 <a href="/detail/101" class="detail">详情</a></li>\n        <li class="item" data-id="102">香蕉 <a href="/detail/102" class="detail">详情</a></li>\n        <li class="item" data-id="103">梨子 <a href="/detail/103" class="detail">详情</a></li>\n      </ul>\n      <p class="desc">水果列表示例，用于 CSS Selector 练习。</p>\n      <div class="nested">\n        <span class="note">附注 A</span>\n        <span class="note">附注 B</span>\n      </div>\n    </div>\n    <footer>Copyright <span>2025</span></footer>\n  </body>\n</html>\n'''
print(sample_html.splitlines()[0], '... 共', len(sample_html.splitlines()), '行')

 ... 共 20 行


## 3. parsel (Scrapy 常用)
`parsel.Selector` 提供 `.css()` 和 `.xpath()` 双接口。CSS 中获取文本和属性使用伪选择器：`::text` 与 `::attr(name)`。适合快速统一写法。

In [5]:
from parsel import Selector
sel = Selector(text=sample_html)
# 选取所有 li.item 的纯文本 (包含子节点文本汇总)
items_text = sel.css('li.item::text').getall()  # 只取 li 直接文本 (不含 <a> 内文)
items_full = [li.get().strip() for li in sel.css('li.item')]
print('items_text:', items_text)
print('items_full 示例:', items_full[0][:40], '...')
# 选取激活项 data-id
active_id = sel.css('li.item.active::attr(data-id)').get()
print('active id:', active_id)
# 获取所有详情链接 href
detail_hrefs = sel.css('a.detail::attr(href)').getall()
print(detail_hrefs)
# 组合选择器：ul.items > li:first-child
first_li_text = sel.css('ul.items > li:first-child::text').get()
print('first li text:', first_li_text)
# 后代 + 属性前缀匹配
links_prefix = sel.css('div.container a[href^="/detail"]::attr(href)').getall()
print('prefix matched hrefs:', links_prefix)

items_text: ['苹果 ', '香蕉 ', '梨子 ']
items_full 示例: <li class="item active" data-id="101">苹果 ...
active id: 101
['/detail/101', '/detail/102', '/detail/103']
first li text: 苹果 
prefix matched hrefs: ['/detail/101', '/detail/102', '/detail/103']


## 4. requests + parsel 抓取在线页面 (可选)
运行需要网络，示例使用 `quotes.toscrape.com`。若离线环境，可跳过。

In [ ]:
import requests
try:
    r = requests.get('https://quotes.toscrape.com/', timeout=10)
    r.raise_for_status()
    page = Selector(text=r.text)
    quotes = page.css('div.quote')
    for q in quotes[:3]:
        text = q.css('span.text::text').get()
        author = q.css('small.author::text').get()
        tags = q.css('div.tags a.tag::text').getall()
        print(author, ':', text[:30], '...', tags)
except Exception as e:
    print('网络访问失败，跳过示例 ->', e)

## 5. BeautifulSoup (select)
`BeautifulSoup` 支持 `.select()` 返回元素列表，以及 `.select_one()`。获取属性用 `el['attr']`，文本用 `.get_text(strip=True)`。伪类支持有限。

In [7]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(sample_html, 'lxml')
li_nodes = soup.select('ul.items > li.item')
print('总数:', len(li_nodes))
print('第一个文本:', li_nodes[0].get_text(strip=True))
active = soup.select_one('li.item.active')
print('active data-id:', active['data-id'])
hrefs = [a['href'] for a in soup.select('a.detail')]
print('hrefs:', hrefs)

总数: 3
第一个文本: 苹果详情
active data-id: 101
hrefs: ['/detail/101', '/detail/102', '/detail/103']


## 6. PyQuery
`pyquery` 语法类似 jQuery：链式、`items()`、`text()`、`attr()`。非常适合熟悉前端的开发者。

In [8]:
from pyquery import PyQuery as pq
doc = pq(sample_html)
for i, li in enumerate(doc('ul.items > li').items(), 1):
    print(i, li.text(), 'data-id=', li.attr('data-id'))
# 直接取所有链接 href
print(doc('a.detail').attr('href'))  # 第一个
print([a.attr('href') for a in doc('a.detail').items()])

1 苹果 详情 data-id= 101
2 香蕉 详情 data-id= 102
3 梨子 详情 data-id= 103
/detail/101
['/detail/101', '/detail/102', '/detail/103']


## 7. lxml 原生 + cssselect
`lxml` 可用 `from lxml import html`，解析后调用 `.cssselect(selector)`。需确保已安装 `cssselect` 包 (通常随 lxml 一起被支持)。

In [9]:
from lxml import html
tree = html.fromstring(sample_html)
li_nodes = tree.cssselect('ul.items > li.item')
print('数量:', len(li_nodes))
print('第一个去除多余空白文本:', ''.join(li_nodes[0].text_content().split()))
# 属性过滤 + 前缀匹配
href_nodes = tree.cssselect('a.detail')
print([n.get('href') for n in href_nodes])

数量: 3
第一个去除多余空白文本: 苹果详情
['/detail/101', '/detail/102', '/detail/103']


## 8. selectolax (性能更好)
`selectolax` 使用基于 Modest engine 的解析，速度快、占用低。CSS 支持子集 (不含复杂伪类)。适合大批量页面。

In [10]:
from selectolax.parser import HTMLParser
parser = HTMLParser(sample_html)
for li in parser.css('ul.items > li.item'):
    # text() 返回拼接文本；attributes 字典
    print(li.attributes.get('data-id'), li.text(strip=True))
links = [a.attributes.get('href') for a in parser.css('a.detail')]
print('links:', links)

101 苹果详情
102 香蕉详情
103 梨子详情
links: ['/detail/101', '/detail/102', '/detail/103']


## 9. 常见技巧与坑
1. 多类匹配：`.item.active` 是同时具有两个类；包含任一类要分两次或用 XPath。
2. 空格 vs `>`：`div a` 后代任意深度；`div > a` 仅直接子代。
3. 文本获取差异：`parsel` 用 `::text` 仅取直接文本节点；`BeautifulSoup` 的 `.get_text()` 合并所有后代。需要精准分离时用 `parsel` 或 `lxml`。
4. 属性选择性能：前缀/后缀匹配 (`^=`, `$=`, `*=`) 在大 DOM 上较慢，可先粗选再过滤。
5. 不支持的伪类：很多库不支持诸如 `:nth-of-type(odd)` 等高级选择器；必要时回退 XPath。
6. 清理文本：多余空白/换行可用 `.strip()`、`re.sub(r'\s+', ' ', text)`。
7. 网络请求：频繁解析建议复用 Session，减少 TLS 握手开销。
8. 编码：确保 `response.encoding` 正确，否则解析文本可能乱码。
9. 性能基线：批量爬取时优先 `selectolax` 或 `lxml`，`BeautifulSoup` 更易用但慢。

## 10. 小型速查表
| 目标 | CSS 示例 | 说明 |
|-------|-----------|------|
| 所有 li | `li` | 标签名 |
| class=detail 的 a | `a.detail` | 类匹配 |
| id=main 的 div | `#main` | id |
| ul.items 下直接子 li | `ul.items > li` | 子代 |
| 任意后代 a | `#main a` | 后代 |
| 有 href 属性的 a | `a[href]` | 属性存在 |
| href 以 /detail 开头 | `a[href^=/detail]` | 前缀 |
| href 以 .png 结尾 | `img[src$=.png]` | 后缀 |
| data-id 含 10 | `li[data-id*=10]` | 子串 |
| 第一个 li | `li:first-child` | 伪类 |
| 第2个 li | `li:nth-child(2)` | 序号 |

> 更多：MDN CSS Selectors 文档可作为权威参考。

## 12. 总结
- CSS 选择器语义直观，适合快速定位元素。
- 各库差异主要体现在：API 风格、性能、伪类支持范围。
- 与 XPath 结合使用可覆盖复杂场景。
- 批量采集关注性能与内存，优先选择高性能解析器。

可根据项目需求选取：`parsel` (Scrapy 生态)、`selectolax` (性能)、`BeautifulSoup` (易用)、`pyquery` (前端风格)、`lxml` (底层灵活)。